In [ ]:
VALID_PTH = "zindi_data/ValDataset.csv"
PRDICTION_PTH = 'zindi_data/validation/prediction_cond-detr-50_THR0.000_IOU0.400_ID580.csv'
THR_INF = 0.01
from zindi_code import CLS_MAPPER
CLS_MAPPER["NEG"] = 2
CLS_MAPPER

In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm

import numpy as np

In [ ]:
valid_data = pd.read_csv(VALID_PTH)

predictions = pd.read_csv(PRDICTION_PTH)
predictions.sample(3)

In [ ]:
valid_data.sample(3)

In [ ]:
# valid_data = valid_data.rename(columns={
# 	"Image_ID": "image_id", "class": "category_id", "confidence": "score", "xmin": "x", "ymin": "y"
# })

# valid_data["w"] = valid_data["xmax"] - valid_data["x"]
# valid_data["h"] = valid_data["ymax"] - valid_data["y"]


# valid_data.sample(3)

In [ ]:
predictions["Image_ID"].nunique(), valid_data["Image_ID"].nunique()

In [ ]:
len(set(predictions["Image_ID"].unique()).intersection(valid_data["Image_ID"].unique()))

In [ ]:
predictions["Image_ID"].value_counts().head(10)

In [ ]:
valid_data["Image_ID"].value_counts().head(10)

In [ ]:
predictions["confidence"].describe()

In [ ]:
predictions.isna().sum()

In [ ]:
len(valid_data[valid_data["class"] == "NEG"])

In [ ]:
predictions.columns

In [ ]:
valid_data.columns

In [ ]:
predictions.sample(5)

In [ ]:
import torch

image_files: list[str] = valid_data["Image_ID"].unique().tolist()


def get_format_pred(file_id: str, data: pd.DataFrame, is_prediction, thr=THR_INF):
    pred = data[data["Image_ID"] == file_id].copy()
    pred = pred[pred["class"] != "NEG"]
    pred = pred[pred["confidence"] >= thr]
    if not len(pred):
        return dict(
        	boxes=torch.empty(size=[0, 4], dtype=torch.float).cuda(),
        	scores=torch.empty(size=[0], dtype=torch.float).cuda(),
        	labels=torch.empty(size=[0,], dtype=torch.int).cuda(),
        )
        return dict(
            boxes=torch.tensor([[0.0, 0.0, 0.0, 0.0]], dtype=torch.float).cuda(),
            scores=torch.tensor([1.0], dtype=torch.float).cuda(),
            labels=torch.tensor([CLS_MAPPER["NEG"]], dtype=torch.int).cuda(),
        )
    pred["w"] = pred["xmax"] - pred["xmin"]
    pred["h"] = pred["ymax"] - pred["ymin"]
    boxes = torch.tensor(
        pred[["xmin", "ymin", "w", "h"]].values, dtype=torch.float
    ).cuda()

    scores = torch.tensor(pred["confidence"].values, dtype=torch.float).cuda()
    labels = torch.tensor(
        [CLS_MAPPER[i] for i in pred["class"].values], dtype=torch.int
    ).cuda()
    result = dict(boxes=boxes, scores=scores, labels=labels)
    return result


print("Prediction Extraction")
preds = [get_format_pred(i, predictions, True) for i in image_files]
print("Expected Extraction")
expected = [get_format_pred(i, valid_data, False) for i in image_files]

In [ ]:
preds[0]

In [ ]:
expected[0]

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    # backend="faster_coco_eval",
)


metric.update(preds=preds, target=expected)

from pprint import pprint

pprint(metric.compute())

In [ ]:
expected[128]

In [ ]:
preds[128]

In [ ]:
from tqdm import tqdm

thrs = np.linspace(0.0, 0.95, num=15)
best_score = 0
best_thr = 0

for thr in tqdm(thrs):
    metric = MeanAveragePrecision(
        iou_type="bbox",
        box_format="xywh",
        class_metrics=True,
        extended_summary=False,
        # backend="faster_coco_eval",
    )
    preds = [get_format_pred(i, predictions, True, thr) for i in image_files]
    expected = [get_format_pred(i, valid_data, False, thr) for i in image_files]
    metric.update(preds=preds, target=expected)
    scores = metric.compute()
    if scores["map_50"] > best_score:
        best_score = scores["map_50"]
        best_thr = thr
        print(best_thr, best_score)

In [ ]:
print(best_thr, best_score)

In [ ]:
thrs

In [ ]:
thr = 0.05

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    backend="faster_coco_eval",
)
preds = [get_format_pred(i, predictions, True, thr) for i in image_files]
expected = [get_format_pred(i, valid_data, False, thr) for i in image_files]
metric.update(preds=preds, target=expected)
scores = metric.compute()

In [ ]:
scores